In [1]:
import pandas as pd
import xlsxwriter
import os
from datetime import datetime

# find the files neede to process
ok_file = ""
cancel_file = ""
booking_file = ""
files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
    f = f.lower()
    if "ok" in f and f[:2] != "~$":
        ok_file = f
    if "cancel" in f and f[:2] != "~$":
        cancel_file = f
    if "expedia" in f and f[:2] != "~$":
        booking_file = f

msg = "Files detected:\n" + booking_file + "\n" + ok_file + "\n" + cancel_file + "\n" + "Press Enter key to begin the process."
a = input(msg)


Files detected:
expedia reservationlist 8.1-8.31.xlsx
30031-guestlistdetailed 8.1-31 ok.xls
30031-guestlistdetailed cancel 8.1-8.31.xlsx
Press Enter key to begin the process.


In [2]:

# developed with python 3.5 for pyinstaller function
#############################################################
# check conditions and print out
def same_name(booking_name, hotel_name):
    if hotel_name[:5] == 'name\n':
        hotel_name = hotel_name[5:]

    hotel_name = hotel_name.split(",")
    hotel_name = hotel_name[1][1:] + " " + hotel_name[0]

    if hotel_name == booking_name:
        # print("h:", hotel_name, "b:", booking_name)
        return True
    else:
        return False


def calc_hotel_price(hotel_price, arrival, departure):
    # print("input: ", booking_price, hotel_price, arrival, departure)

    if departure[:2] == "De":
        departure = departure[9:]
        arrival = arrival[7:]
        hotel_price = hotel_price[4:]

    date_format = "%m/%d/%Y"
    a = datetime.strptime(departure, date_format)
    b = datetime.strptime(arrival, date_format)
    delta = a - b
    # print("hotel price: ", float(delta.days) * float(hotel_price), "booking price: ", float(booking_price))
    # print(float(delta.days) * float(hotel_price) == float(booking_price))
    return (float(delta.days) * float(hotel_price))




In [6]:
match = 0
diff_price = 0
canceled = 0
not_found = 0

# load and prepare the booking df
xl = pd.ExcelFile(booking_file)
booking_df = xl.parse(header=0, keep_default_na=False)

booking_col_names = list(booking_df)
print(booking_col_names)
for name in booking_col_names:
    a = name.split(" ")
    if a[-1] == "":
        b = " ".join(a[:-1])
    else:
        b = " ".join(a)
    booking_df = booking_df.rename(columns={name: b})

booking_col_names = list(booking_df)
print(booking_col_names)

b_df = booking_df[booking_df['Status'] == 'Booked']
print('booking df created')
# for index, row in b_df.iterrows():
#     if row['Guest name(s)'] == '':
#         name = str(row['Booked by'])
#         guest_name = name.split(",")
#         b_df.loc[index, 'Guest name(s)'] = guest_name[1][1:] + " " + guest_name[0]



['Reservation ID', 'Guest', 'Check-In', 'Check-Out', 'Reservation Made', 'Room', 'Who Collects', 'Confirmation #', 'Status']
['Reservation ID', 'Guest', 'Check-In', 'Check-Out', 'Reservation Made', 'Room', 'Who Collects', 'Confirmation #', 'Status']
booking df created


In [7]:
b_df

,Reservation ID,Guest,Check-In,Check-Out,Reservation Made,Room,Who Collects,Confirmation #,Status
0,789180681,Holly Biffin,2017-08-03,2017-08-05,2017-02-05T11:33:00-08:00,"Standard Room, 1 Queen Bed, Non Smoking",hotel,84974500,Booked
1,815263048,Michelle DaCosta,2017-07-29,2017-08-01,2017-03-26T00:31:00-07:00,"Standard Room, 1 Queen Bed, Non Smoking",hotel,13298521,Booked
2,840882614,Mercedes Rodriguez,2017-08-24,2017-08-26,2017-05-12T17:24:00-07:00,"Room, 2 Double Beds, Non Smoking",hotel,32899954,Booked
3,844018524,JANA KONO,2017-08-22,2017-08-24,2017-05-19T09:37:00-07:00,"Room, 2 Double Beds, Non Smoking",hotel,35255132,Booked
4,844186331,Sheldon Kaye,2017-08-01,2017-08-03,2017-05-19T15:40:00-07:00,"Standard Room, 1 Queen Bed, Non Smoking",hotel,35449783,Booked
5,849211454,Martin Dickey,2017-08-14,2017-08-18,2017-05-29T13:05:00-07:00,"Standard Room, 1 Queen Bed, Non Smoking",hotel,39686312,Booked
6,857401553,richard thorpe,2017-08-06,2017-08-10,2017-06-13T11:48:00-07:00,"Room, 2 Double Beds, Non Smoking",hotel,46329924,Booked
7,863838405,Frederike Driesse,2017-08-07,2017-08-09,2017-06-24T22:01:00-07:00,"Room, 2 Double Beds, Non Smoking",hotel,51832537,Booked
8,864709221,Shirly E. Sullivan,2017-08-21,2017-08-22,2017-06-26T12:53:00-07:00,"Room, 2 Double Beds, Non Smoking",hotel,52443024,Booked
9,864903593,Camille a Stone,2017-08-23,2017-08-28,2017-06-26T19:11:00-07:00,"Room, 2 Double Beds, Non Smoking",hotel,52650765,Booked


In [8]:
##########################################
# loading the ok DF
xl = pd.ExcelFile(ok_file)
ok_df = xl.parse("Sheet1", header=2, keep_default_na=False, )
print(list(ok_df))
o_df = ok_df.loc[(ok_df['Name'] != "") & (ok_df['Name'] != "Name") &
                 (ok_df['Name'] != "Guest List Detail") &
                 (ok_df['Name'] != "30031 - DAYS INN PHILADELPHIA.") &
                 (ok_df['Name'] != "Name Arrival Departure Rate")]

# combine ok name
for row in range(1, len(o_df)):
    if o_df.iloc[row]['Rate'] == '':
        o_df.iloc[row - 1]['Name'] = o_df.iloc[row - 1]['Name'] + " " + o_df.iloc[row]['Name']

# clear all empty rows and finalize df
o_df = o_df.loc[(o_df['Rate'] != "")]

# print o_df
# for index, row in o_df.iterrows():
#     print("index:::", index, row['Name'], row['Arrival'], row['Departure'], row['Rate'])
#

##########################
# loading the cancel DF
xl = pd.ExcelFile(cancel_file)
c_df = xl.parse("Sheet1", header=2, keep_default_na=False, )
print(list(c_df))
c_df = c_df.loc[(c_df['Name'] != "") & (c_df['Name'] != "Name") &
                (c_df['Name'] != "Guest List Detail") &
                (c_df['Name'] != "30031 - DAYS INN PHILADELPHIA.") &
                (c_df['Name'] != "Name Arrival Departure Rate")]

# combine ok name
for row in range(1, len(c_df)):
    if c_df.iloc[row]['Rate'] == '':
        c_df.iloc[row - 1]['Name'] = c_df.iloc[row - 1]['Name'] + " " + c_df.iloc[row]['Name']

# clear all empty rows and finalize df
c_df = c_df.loc[(c_df['Rate'] != "")]

# print o_df
# for index, row in c_df.iterrows():
#     print("index:::", index, row['Name'], row['Arrival'], row['Departure'], row['Rate'])

ok_file = ""
cancel_file = ""
booking_file = ""
files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
    if " ok " in f and f[:2] != "~$":
        ok_file = f
    if " cancel " in f and f[:2] != "~$":
        cancel_file = f
    if " booking " in f and f[:2] != "~$":
        booking_file = f



WARNING *** file size (1635961) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
['Name', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Level', 'Unnamed: 8', 'Unnamed: 9', 'Room Type', 'Unnamed: 11', 'Unnamed: 12', 'Room', 'Source', 'Unnamed: 15', 'Unnamed: 16', 'Acct#', 'Unnamed: 18', 'Unnamed: 19', 'CRS#', 'Unnamed: 21', 'Unnamed: 22', 'Status', 'Unnamed: 24', 'Arrival', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Departure', 'Unnamed: 30', 'Unnamed: 31', 'Adult/Child', 'GTD', 'Unnamed: 34', 'Rate', 'Unnamed: 36', 'Unnamed: 37', 'Rate Plan', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Balance', 'Unnamed: 43']
['Name', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Level', 'Unnamed: 7', 'Unnamed: 8', 'Room Type', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Room', 'Source', 'Unnamed: 15', 'Unnamed: 16', 'Acct#', 'Unnamed: 18', 'Unnamed: 19', 'CRS#', 'Unna

In [9]:
o_df

,Name,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Level,Unnamed: 8,Unnamed: 9,...,Unnamed: 34,Rate,Unnamed: 36,Unnamed: 37,Rate Plan,Unnamed: 39,Unnamed: 40,Unnamed: 41,Balance,Unnamed: 43
,"ABASCAL, CLOTILDE",,,,,,,,,,...,,$129.00,,,SBK,,,,,$0.00
,"ACEVEDO, FRANCISCO",,,,,,,,,,...,,$75.00,,,LNY2,,,,,$0.00
,"ACHING, JOAN",,,,,,,,,,...,,$135.20,,,SBK,,,,,$0.00
,"ADAMS, DANE",,,,,,,,,,...,,$69.00,,,LNY2,,,,,$0.00
,"ADAMS, DANE",,,,,,,BLUE,,,...,,$87.00,,,LNY2,,,,,$0.00
,"ADEDIWURA, FEYISAYO",,,,,,,,,,...,,$83.00,,,SOE,,,,,$0.00
,"ADUSARKODIE, SETH",,,,,,,,,,...,,$100.73,,,RACK,,,,,$0.00
,"AGRAWAL, TUSHAR",,,,,,,,,,...,,$71.40,,,LNY2,,,,,$0.00
,"AHMED, STYED",,,,,,,,,,...,,$119.00,,,RACK,,,,,$0.00
,"AL HAWAISH, BADR",,,,,,,,,,...,,$135.20,,,SBK,,,,,$0.00


In [10]:
c_df

,Name,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Level,Unnamed: 7,Unnamed: 8,Room Type,...,Unnamed: 33,Rate,Unnamed: 35,Unnamed: 36,Rate Plan,Unnamed: 38,Unnamed: 39,Balance,Unnamed: 41,Unnamed: 42
,"ABD-ERRAZIK, MOHAMMAD",,,,,,,,,,...,,$77.49,,,SBK,,,,$0.00,
,"AHMED, SYED",,,,,,,,,,...,,$114.00,,,SBK2,,,,$0.00,
,"ALI, MOHAMMAD",,,,,,,,,,...,,$83.95,,,SOE,,,,$0.00,
,"ALI, MOHAMMAD",,,,,,,,,,...,,$135.00,,,RACK,,,,$0.00,
,"ALI, MOHAMMAD",,,,,,,,,,...,,$73.37,,,SDO,,,,$0.00,
,"ALLEN, ANGELICA",,,,,,,,,,...,,$113.65,,,SBK2,,,,$0.00,
,"ALLEN, MICHELLE",,,,,,,,,,...,,$75.70,,,SDO,,,,$0.00,
,"Name\nALNAHARI, AMANI",,,,,,Level,,,Room TypeNQ1,...,,Rate$101.20,,,Rate PlanSBK,,,Balance,$0.00,
,"ALONZO, ANTHONY",,,,,,,,,,...,,$151.20,,,SOEP,,,,$0.00,
,"ALQURAINI, AHMAD",,,,,,,,,,...,,$92.00,,,SOEP,,,,$0.00,


In [12]:
b_df['Guest'][0]

'Holly Biffin'

In [24]:
for index_ok, row_ok in o_df.iterrows():
    print(row_ok['Name'])

ABASCAL, CLOTILDE
ACEVEDO, FRANCISCO
ACHING, JOAN
ADAMS, DANE
ADAMS, DANE
ADEDIWURA, FEYISAYO
ADUSARKODIE, SETH
AGRAWAL, TUSHAR
AHMED, STYED
AL HAWAISH, BADR
ALAMRI, TAREK
ALAMRI, TAREK
ALASALEH, MOHAMAD
ALESSIA, GIULIANO
ALLEN, CORNELL
ALLEN, DOMINIQUE
ALNAHARI, AMANI
ALOTAIBI, HAMOUD
ALQOBLAN, YASER
ALSAAD, EBRAHEEM
ALSAAD, EBRAHEEM
ALSAAD, EBRAHEEM
ALSAAD, EBRAHEEM
ALSADAH, ZAINAB
ALVARELLOS JESUS,ALBERTO SUAREZ
ALZUABI, FAHAD
AMICI, LENE
ANDERSON, JAMIE
ANDERSON, MARC
ANDERSON, MARC
ANDERSON, THEODORE
Name
ANDRE, MUNOZ
ANGEL LEON COTE, MIGUEL
ANSLEY, BONNIE
ARANDA, CYNTHIA
ARIAS, MARIA
ARNAU LAZARO, SARA
ASHENFELTER, RICHARD BRUCE
Name
ASQUITH, PHILIP
AUSTRIA, VANESSA
AVTAR, KRISHAN
BAILEY, BRODERICK
BAILLEUL, MARIE- EMMANUELLE
BAKER, KURT
BALLABEN, ANELISE S
Name
BARKEY, CONOR
BARRIENTOS, FRANZ
BARRON, CAROLYN
BARRON, CAROLYN
BARTOS, VOJTECH
BASSETT, DENTON WAYNE
BASTA, ROBERT
Name
BASZYNSKI, GREGORY
BATTLE, PATRICK
BEASON, STEPHANIE M
BEDARD, DENIS
BEDASCARRASBURE, OMAR
BELEGUM, 

In [25]:
for index, row in b_df.iterrows():
    print(row['Guest'])

Holly Biffin
Michelle DaCosta
Mercedes Rodriguez
JANA KONO
Sheldon Kaye
Martin Dickey
richard thorpe
Frederike Driesse
Shirly E. Sullivan
Camille a Stone
William Jerry Rodgers
Jean Loïc Brette
Mitchell stephens
Thomas Hurwitz
Ji Yue
Veronica Fleming
Ertha Dean
Cristian Rojas
Liz Stroppel
Wenzhu Wang
Debie Hock
Chantel Carter
Clarence Navarro
Mirren Cormack
Jacqueline Ketchens
Christy Pflanzer
susan Kacin-Haas
Bernard Rogan
Bryan Pennington
Clarissa Corbin
Jessica Feltz
laura marsh
Dan W Martin
Kevin Gross
William Zimmer
Raymond Richard
Kathy Faber-Pitts
Marco Butera
thomas carroll
Maciej Szulepa
Jean Morales
Athalia Montaño
Cynthia Aranda
Nathan Smith
Matthew McCutcheon
Cecilie Mottlau
Olivier Lefrançois
Chafarrah Clemons
Kiana Spady
Billy Jack
Javon Parkes
Alexander Gilroy
Abhijay Shekhawat
Chafarrah Clemons
Carlos Rocha
Milton Owens, Jr.
Brian McMonagle
Dawne Marie Pound
Nazir Ninson
Chinyearra Monroe
Ismet deger EPOZDEMIR
Thomas Cook Jr
Patrick McCarthy
Michael Ritter
Shoji Kunitomo

In [32]:
for index, row in b_df.iterrows():

    # initialzie what to compare
    name = row['Guest']
    check_in_date = row['Check-In']
    check_out_date = row['Check-Out']
    # print("check_in_date: ", check_in_date)
    check_in_date = str(check_in_date).split("-")
    # print("check_in_date after split: ", check_in_date)
    check_out_date = str(check_out_date).split("-")
    # price = row['Price'][:-3]
    found = False

    # check if in ok, but different price
    print('start checking loop')
    for index_ok, row_ok in o_df.iterrows():
        if (same_name(name.lower(), row_ok['Name'].lower())):
            print('Name matches!!!!!!!!!!!!!!!!!!')
            if row_ok['Departure'][:2] == "De":
                departure = row_ok['Departure'][9:]
                arrival = row_ok['Arrival'][7:]
            else:
                departure = row_ok['Departure']
                arrival = row_ok['Arrival']
            # hotel date
            date_format = "%m/%d/%Y"
            h_departure = datetime.strptime(departure, date_format)
            h_arrival = datetime.strptime(arrival, date_format)

            # booking.com date

            b_checkin = datetime(int(check_in_date[0]), int(check_in_date[1]), int(check_in_date[2]))
            b_checkout = datetime(int(check_out_date[0]), int(check_out_date[1]), int(check_out_date[2]))

            if (h_arrival == b_checkin) & (h_departure == b_checkout):
                found = True
                hotel_price = calc_hotel_price(row_ok['Rate'][1:], row_ok['Arrival'], row_ok['Departure'])
                price_difference = hotel_price - float(price)
                if price_difference != 0.0:

                    print(price_difference)
                    print(total_price_difference)
                    print(name, price, "found in Ok file ,but different price")
                    worksheet.write(rowm, coln, name)
                    worksheet.write(rowm, coln + 1, price)
                    worksheet.write(rowm, coln + 2, hotel_price)
                    worksheet.write(rowm, coln + 3, price_difference)
                    total_price_difference += price_difference
                    worksheet.write(rowm, coln + 4, "Found in OK file, but price is different.")
                    rowm += 1
                    diff_price += 1
                else:
                    match += 1
                    print(name, ' Everything match')

            break

    if (found == False):
        # check if in canceled
        for index_c, row_c in c_df.iterrows():
            if (same_name(name.lower(), row_c['Name'].lower())):
                # print(name + "**BAD***********************************************")
                # worksheet.write(rowm, coln, name)
                #
                # worksheet.write(rowm, coln + 4, "Found in Cancel file.")
                # rowm += 1
                # found = True
                # canceled += 1

                if row_c['Departure'][:2] == "De":
                    departure = row_c['Departure'][9:]
                    arrival = row_c['Arrival'][7:]
                else:
                    departure = row_c['Departure']
                    arrival = row_c['Arrival']
                    # hotel date
                date_format = "%m/%d/%Y"
                h_departure = datetime.strptime(departure, date_format)
                h_arrival = datetime.strptime(arrival, date_format)

                # booking.com date

                b_checkin = datetime(int(check_in_date[0]), int(check_in_date[1]), int(check_in_date[2]))
                b_checkout = datetime(int(check_out_date[0]), int(check_out_date[1]), int(check_out_date[2]))

                if (h_arrival == b_checkin) & (h_departure == b_checkout):
                    found = True
                    hotel_price = calc_hotel_price(row_ok['Rate'][1:], row_ok['Arrival'], row_ok['Departure'])
                    price_difference = hotel_price - float(price)

                    print(name, price, "found in CANCEL file")
                    worksheet.write(rowm, coln, name)
                    worksheet.write(rowm, coln + 1, price)
                    worksheet.write(rowm, coln + 2, hotel_price)
                    worksheet.write(rowm, coln + 3, price_difference)
                    total_price_difference += price_difference
                    worksheet.write(rowm, coln + 4, "found in CANCEL file, but price is different.")
                    rowm += 1
                    diff_price += 1

                    break

    if found == False:
        print(name, " Can't find customer Name")
        not_found += 1
        worksheet_nf.write(row_nf, 0, name)
        row_nf += 1
print(total_price_difference)
worksheet.write(rowm, 2, "Total difference: ->")
worksheet.write(rowm, 3, total_price_difference)
workbook.close()
workbook_nf.close()

print("match: ", match)
print("diff_price: ", diff_price)
print("canceled: ", canceled)
print("not found: ", not_found)


start checking loop
Name matches!!!!!!!!!!!!!!!!!!


ValueError: invalid literal for int() with base 10: '03 00:00:00'

In [31]:
b_df['Check-In']

0     2017-08-03
1     2017-07-29
2     2017-08-24
3     2017-08-22
4     2017-08-01
5     2017-08-14
6     2017-08-06
7     2017-08-07
8     2017-08-21
9     2017-08-23
10    2017-08-02
11    2017-08-08
12    2017-08-23
13    2017-08-01
14    2017-08-06
15    2017-07-30
16    2017-07-30
17    2017-07-31
18    2017-07-31
19    2017-07-31
20    2017-08-03
21    2017-08-02
22    2017-07-31
23    2017-08-03
24    2017-08-21
25    2017-08-02
26    2017-08-30
27    2017-08-27
28    2017-08-27
29    2017-07-31
         ...    
79    2017-08-29
80    2017-08-27
81    2017-08-18
82    2017-08-23
83    2017-08-25
84    2017-08-17
85    2017-08-20
86    2017-08-16
87    2017-08-16
88    2017-08-28
89    2017-08-12
90    2017-08-21
91    2017-08-24
92    2017-08-26
93    2017-08-28
94    2017-08-25
95    2017-08-25
96    2017-08-25
97    2017-08-25
98    2017-08-26
99    2017-08-27
100   2017-08-24
101   2017-08-24
102   2017-08-22
103   2017-08-23
104   2017-08-24
105   2017-08-21
106   2017-08-